In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code for QISKit exercises."></form>''')

# 10. Quantum error correction

In the last chapter, we saw how the impossibility of copying an unknown quantum state adds a layer of security to quantum communication when compared to classical ones. At first sight, this feature turns into a drawback once we step into the realm of error correction. In fact, classically one can protect information by making many copies of the bits of interest. The probability that an error will happen to all the copies quickly becomes extremely unlikely with an increasing number of copies. However, the no-cloning theorem prohibits us from using the same technique for quantum bits.
Interestingly, error correcting procedure exists also in quantum computation. In this case, quantum entanglement and quantum superposition are used as resources to fight against unwanted errors. In this chapter, we take a closer look at quantum error correction for the simple cases of a bit-flip error and a phase error following a five-qubit protocol outlined in Ref. [1].

## 10.1 The bit-flip error in the three-bit code

<img src="figures/10/bit_flip2.jpeg"  width="950">
$$\text{1. Error correcting circuit for a bit-flip error.}$$

Let us analyze the case of transmission of quantum information, in the form of a qubit, through a noisy channel. Because of the presence of noise, an error could occur during the communication which could change the state of the qubit in an unpredictable way. We must be able to restore its state without measuring the qubit.


Let us make a few assumptions. First of all, we are going to assume that if more than one qubits are transferred, errors happen independently on each qubit. Also, here we are going to consider only single bit-flip ($X$) errors.

Take the qubit we want to transfer to be in the state

\begin{equation}
\lvert \psi \rangle= \alpha\lvert 0\rangle +\beta \lvert 1\rangle
\end{equation}

As a first step, we encode this qubit in a three-qubit entangled state using two extra ancilla qubits in the state $\lvert 0 \rangle$ and CNOT gates from the first qubits to the ancillas ($CX_{12}CX_{13}$)

\begin{equation}
\lvert\psi_1\rangle=\alpha\lvert000\rangle+\beta\lvert110\rangle
\end{equation}


The three qubits are then sent through the noisy communication channel and retrieved on the receiver's side. During the communication, a bit-flip error could have occurred on any of the three qubits giving one of the following states

<ol>
<li>
$\lvert \psi_{2} \rangle =\alpha\lvert 000\rangle+\beta\lvert 111\rangle$
</li>

<li>
$\lvert \psi_{2} \rangle =\alpha\lvert100\rangle+\beta\lvert011\rangle$
</li>

<li>
$\lvert\psi_{2} \rangle =\alpha\lvert010\rangle+\beta\lvert101\rangle$
</li>

<li>
$\lvert\psi_{2} \rangle =\alpha\lvert001\rangle+\beta\lvert110\rangle$
</li>

</ol>


In order to correct the state received at the other end of the quantum communication channel from possible errors, two more ancilla qubits in the state $\lvert 0\rangle$ are added to the system. Moreover, they are entangled with the three bit received by using the following gates: $CX_{14}CX_{24}CX_{15}CX_{35}$. The state of the system is now


<ol>
<li>
$\lvert\psi_{3} \rangle =\alpha \lvert000 \rangle \lvert 00\rangle+\beta\lvert111 \rangle \lvert 00\rangle$
</li>

<li>
$\lvert\psi_{3} \rangle =\alpha\lvert001 \rangle \lvert 01\rangle+\beta\lvert110 \rangle \lvert 01\rangle$
</li>

<li>
$\lvert\psi_{3} \rangle =\alpha\lvert010 \rangle \lvert 10\rangle+\beta\lvert101 \rangle \lvert 10\rangle$
</li>

<li>
$\lvert\psi_{3} \rangle =\alpha\lvert100 \rangle \lvert 11\rangle+\beta\lvert011 \rangle \lvert 11\rangle$
</li>

</ol>

The two ancilla qubits just added are then measured. They give the "error syndrom", which can be used to diagnose and then correct any error occurred on the three qubits. There are four possible outcomes for the error syndrome:


<ol>
<li>
$00$: $\lvert\psi_{4} \rangle =\alpha \lvert000 \rangle +\beta\lvert111 \rangle $
</li>

<li>
$01$: $\lvert\psi_{4} \rangle =\alpha\lvert001 \rangle +\beta\lvert110 \rangle$
</li>

<li>
$10$: $\lvert\psi_{4} \rangle =\alpha\lvert010 \rangle +\beta\lvert101 \rangle$
</li>

<li>
$11$: $\lvert\psi_{4} \rangle =\alpha\lvert100 \rangle +\beta\lvert011 \rangle$

</li>

</ol>

Therefore, the state of the three qubits can be corrected by applying the corresponding gate

<ol>
<li>
$00$: No correction is needed
</li>

<li>
$01$: Apply $X$ gate to the third qubit
</li>

<li>
$10$: Apply $X$ gate to the second qubit
</li>

<li>
$11$: Apply $X$ gate to the first qubit

</li>

</ol>

After which the state of the system will be

\begin{equation}
\lvert\psi_5\rangle=\alpha\lvert000\rangle+\beta\lvert111\rangle
\end{equation}

As originally sent. Now, to find the state of the qubit which was intended to be transferred, we decode it from the three-qubit state by disentangling the qubit with the two ancillas. Thus we apply $CX_{12}CX_{13}$ and obtain

\begin{equation}
\lvert\psi_6\rangle = \alpha\lvert0\rangle+\beta\lvert1\rangle = \lvert\psi \rangle
\end{equation}

We have successfully communicated the state of the qubit $\lvert\psi \rangle$ through a noisy channel, protecting it from any single bit-flip error.

### <span style="color:blue"> QISKit: implement the quantum error correction code for bit-flip error </span>

#### <span style="color:blue"> 1) Correct an X error on the second qubit in the state $ \frac{1}{\sqrt{2}}\left( \lvert 0 \rangle + \lvert 1 \rangle \right)$ </span>

In [6]:
from qiskit import *

# Quantum program setup 
Q_program = QuantumProgram()

# Creating registers
q = Q_program.create_quantum_register('q', 5)
c0 = Q_program.create_classical_register('c0', 1)
c1 = Q_program.create_classical_register('c1', 1)
c2 = Q_program.create_classical_register('c2', 1)

# Creates the quantum circuit
bit_flip = Q_program.create_circuit('bit_flip', [q], [c0,c1,c2])

# Prepares qubit in the desired initial state 
bit_flip.h(q[0])

# Encodes the qubit in a three-qubit entangled state  
bit_flip.cx(q[0], q[1])
bit_flip.cx(q[0], q[2])

# Bit-flip error on the second qubit
bit_flip.x(q[1])

# Adds additional two qubits for error-correction
bit_flip.cx(q[0], q[3])
bit_flip.cx(q[1], q[3])
bit_flip.cx(q[0], q[4])
bit_flip.cx(q[2], q[4])

# Measure the two additional qubits
bit_flip.measure(q[3], c0[0])
bit_flip.measure(q[4], c1[0])

# Do error correction
bit_flip.x(q[1]).c_if(c0, 1)
bit_flip.x(q[2]).c_if(c1, 1)

# Decodes the qubit from the three-qubit entangled state  
bit_flip.cx(q[0], q[1])
bit_flip.cx(q[0], q[2])

# Check the state of the initial qubit
bit_flip.measure(q[0], c2[0])

# Shows gates of the circuit
circuits = ['bit_flip']
print(Q_program.get_qasms(circuits)[0])

# Parameters for execution on simulator
backend = 'local_qasm_simulator' 
shots = 1024 # the number of shots in the experiment 

# Run the algorithm
result = Q_program.execute(circuits, backend=backend, shots=shots)

#Shows the results obtained from the quantum algorithm 
counts = result.get_counts('bit_flip')

print('\nThe measured outcomes of the circuits are:', counts)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c0[1];
creg c1[1];
creg c2[1];
h q[0];
cx q[0],q[1];
cx q[0],q[2];
x q[1];
cx q[0],q[3];
cx q[1],q[3];
cx q[0],q[4];
cx q[2],q[4];
measure q[3] -> c0[0];
measure q[4] -> c1[0];
if(c0==1) x q[1];
if(c1==1) x q[2];
cx q[0],q[1];
cx q[0],q[2];
measure q[0] -> c2[0];


The measured outcomes of the circuits are: {'0 0 1': 501, '1 0 1': 523}


## 10.2 The phase error in the three-bit code

<img src="figures/10/phase_error2.jpeg"  width="1100">
$$\text{2. Error correcting circuit for a phase error.}$$

Consider now the case of a phase error. Phase error transforms the basis states as

\begin{eqnarray}
\lvert 0 &\rangle&\rightarrow U\lvert0\rangle= e^{i\phi}\lvert 0 \rangle  \notag \\
\lvert 1 &\rangle&\rightarrow U\lvert1\rangle=e^{-i\phi}\lvert 1 \rangle
\end{eqnarray}

Let us see how we can protect a qubit in the generic state $\lvert \psi \rangle=\lvert \alpha \lvert0\rangle+ \beta \lvert1\rangle$ from this kind of errors. We are going to assume again that phase error occurs independently on each qubit and that only single phase error may occur.

Before sending the qubit through the noisy channel, we encode it in an entangled state similar to the previous one by using two ancillas in the state $\lvert 0 \rangle$.  To do so we apply $CX_{12}CX_{13}$ and then Hadamard gate on all three qubits, obtaining the encoded state

\begin{eqnarray}
\lvert \psi_1 \rangle = \frac{1}{\sqrt{2^3}}\left\{ \alpha \left[ (\lvert0\rangle+\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle) \right] +  \beta \left[ (\lvert0\rangle-\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle) \right] \right\}
\end{eqnarray}


The three qubits are then sent through the noisy communication channel and retrieved on the receiver's side. During the communication, a phase error could have occurred on any of the three qubits giving one of the following states

<ol>
<li>
$\lvert \psi_{2} \rangle = \frac{1}{\sqrt{2^3}}\left\{ \alpha \left[ (\lvert0\rangle+\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle) \right] +  \beta \left[ (\lvert0\rangle-\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle) \right] \right\}$
</li>

<li>
$\lvert \psi_{2} \rangle = \frac{1}{\sqrt{2^3}}\left\{ \alpha \left[ (e^{i\phi}\lvert0\rangle+e^{-i\phi}\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle) \right] +  \beta \left[ (e^{i\phi} \lvert0\rangle-e^{-i\phi}\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle) \right] \right\}$
</li>

<li>
$\lvert \psi_{2} \rangle =\frac{1}{\sqrt{2^3}}\left\{ \alpha \left[ (\lvert0\rangle+\lvert1\rangle)(e^{i\phi}\lvert0\rangle+e^{-i\phi}\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle) \right] +  \beta \left[ (\lvert0\rangle-\lvert1\rangle)(e^{i\phi}\lvert0\rangle-e^{-i\phi}\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle) \right] \right\}$
</li>

<li>
$\lvert \psi_{2} \rangle = \frac{1}{\sqrt{2^3}}\left\{ \alpha \left[ (\lvert0\rangle+\lvert1\rangle)(\lvert0\rangle+\lvert1\rangle)(e^{i\phi} \lvert0\rangle+e^{-i\phi}\lvert1\rangle) \right] +  \beta \left[ (\lvert0\rangle-\lvert1\rangle)(\lvert0\rangle-\lvert1\rangle)(e^{i\phi} \lvert0\rangle-e^{-i\phi}\lvert1\rangle) \right] \right\}$
</li>

</ol>


In order to correct the state received at the other end of the quantum communication channel from possible errors, we first apply the Hadamard gate on all three qubits again. Giving:

<ol>
<li>
$\lvert \psi_{3} \rangle =  \alpha \lvert000\rangle +  \beta \lvert111\rangle$
</li>

<li>
$\lvert \psi_3 \rangle = \frac{1}{\sqrt{2^3}}\left\{ \alpha \left[   (e^{i\phi}\lvert0\rangle + e^{i \phi} \lvert1\rangle + e^{-i\phi}\lvert0\rangle - e^{-i\phi}\lvert1\rangle) \lvert00\rangle \right] +  \beta \left[ (e^{i\phi}\lvert0\rangle + e^{i\phi}\lvert1\rangle - e^{-i\phi}\lvert 0 \rangle + e^{-i\phi}\lvert 1 \rangle) \lvert11\rangle \right] \right\}$
</li>

<li>
$\lvert \psi_{3} \rangle =\frac{1}{\sqrt{2^3}}\left\{ \alpha \left[ \lvert0\rangle  (e^{i\phi}\lvert0\rangle + e^{i \phi} \lvert1\rangle + e^{-i\phi}\lvert0\rangle - e^{-i\phi}\lvert1\rangle)  \lvert0\rangle \right] +  \beta \left[ \lvert1\rangle  (e^{i\phi}\lvert0\rangle + e^{i \phi} \lvert1\rangle + e^{-i\phi}\lvert0\rangle - e^{-i\phi}\lvert1\rangle)  \lvert1\rangle \right] \right\}$
</li>

<li>
$\lvert \psi_3 \rangle = \frac{1}{\sqrt{2^3}}\left\{ \alpha \left[ \lvert00\rangle  (e^{i\phi}\lvert0\rangle + e^{i \phi} \lvert1\rangle + e^{-i\phi}\lvert0\rangle - e^{-i\phi}\lvert1\rangle) \right] +  \beta \left[ \lvert11\rangle (e^{i\phi}\lvert0\rangle + e^{i\phi}\lvert1\rangle - e^{-i\phi}\lvert 0 \rangle + e^{-i\phi}\lvert 1 \rangle) \right] \right\}$
</li>

</ol>

Expanding the phase as $e^{i \phi} = \cos \phi + i \sin \phi$ and simplifying terms we obtain


<ol>
<li>
$\lvert \psi_{3} \rangle =  \alpha \lvert000\rangle +  \beta \lvert111\rangle$
</li>

<li>
$\lvert \psi_3 \rangle =\alpha(\cos \phi \lvert000 \rangle+i\sin \phi \lvert100\rangle)+\beta(\cos \phi \lvert111\rangle+i\sin \phi \lvert011\rangle)$
</li>

<li>
$\lvert \psi_3 \rangle =\alpha(\cos \phi \lvert000 \rangle+i\sin \phi \lvert010\rangle)+\beta(\cos \phi \lvert111\rangle+i\sin \phi \lvert101\rangle)$
</li>

<li>
$\lvert \psi_3 \rangle =\alpha(\cos \phi \lvert000 \rangle+i\sin \phi \lvert001\rangle)+\beta(\cos \phi \lvert111\rangle+i\sin \phi \lvert110\rangle)$
</li>

</ol>



Two more ancilla qubits in the state $\lvert 0\rangle$ are added to the system. Moreover, they are entangled with the three bit received by using the following gates: $CX_{14}CX_{24}CX_{15}CX_{35}$. The state of the system is now


<ol>
<li>
$\lvert\psi_{4} \rangle =\alpha \lvert000 \rangle \lvert 00\rangle+\beta\lvert111 \rangle \lvert 00\rangle$
</li>

<li>
$\lvert\psi_{4} \rangle =\alpha(\cos \phi \lvert000 \rangle \lvert 00\rangle+i\sin \phi \lvert100 \rangle \lvert 11\rangle)+\beta(\cos \phi \lvert111 \rangle \lvert 00\rangle+i\sin \phi \lvert011 \rangle \lvert 11\rangle)$
</li>

<li>
$\lvert\psi_{4} \rangle =\alpha(\cos \phi \lvert000 \rangle \lvert 00\rangle+i\sin \phi \lvert010 \rangle \lvert 10\rangle)+\beta(\cos \phi \lvert111 \rangle \lvert 00\rangle+i\sin \phi \lvert101 \rangle \lvert 10\rangle)$
</li>

<li>
$\lvert\psi_{4} \rangle =\alpha(\cos \phi \lvert000  \rangle \lvert 00\rangle+i\sin \phi \lvert001 \rangle \lvert 01\rangle)+\beta(\cos \phi \lvert111 \rangle \lvert 00\rangle+i\sin \phi \lvert110 \rangle \lvert 01\rangle)$
</li>

</ol>

The two ancilla qubits just added are then measured. They give the "error syndrome", which can be used to diagnose and then correct any error occurred on the three qubits. There are four possible outcomes for the error syndrome:


<ol>
<li>
$00$: $\lvert\psi_{5} \rangle =\alpha \lvert000 \rangle +\beta\lvert111 \rangle $
</li>

<li>
$11$: $\lvert\psi_{5} \rangle =\alpha\lvert100 \rangle +\beta\lvert011 \rangle$
</li>

<li>
$10$: $\lvert\psi_{5} \rangle =\alpha\lvert010 \rangle +\beta\lvert101 \rangle$
</li>

<li>
$01$: $\lvert\psi_{5} \rangle =\alpha\lvert001 \rangle +\beta\lvert110 \rangle$
</li>

</ol>

Therefore, the state of the three qubits can be corrected by applying the corresponding gate

<ol>
<li>
$00$: No correction is needed
</li>

<li>
$11$: Apply $X$ gate to the first qubit
</li>

<li>
$10$: Apply $X$ gate to the second qubit
</li>

<li>
$01$: Apply $X$ gate to the third qubit

</li>

</ol>

After which the state of the system will be

\begin{equation}
\lvert\psi_6\rangle=\alpha\lvert000\rangle+\beta\lvert111\rangle
\end{equation}

As originally sent. Now, to find the state of the qubit which was intended to be transferred, we decode it from the three-qubit state by disentangling the qubit with the two ancillas. Thus we apply $CX_{12}CX_{13}$ and obtain

\begin{equation}
\lvert\psi_7\rangle = \alpha\lvert0\rangle+\beta\lvert1\rangle = \lvert\psi \rangle
\end{equation}

We have successfully communicated the state of the qubit $\lvert\psi \rangle$ through a noisy channel, protecting it from any single phase error.

### <span style="color:blue"> QISKit: implement the quantum error correction code for phase error </span>

#### <span style="color:blue"> 2) Correct a Z error on the second qubit in the state $ \frac{1}{\sqrt{2}}\left( \lvert 0 \rangle + \lvert 1 \rangle \right)$ </span>

In [7]:
from qiskit import *

# Quantum program setup 
Q_program = QuantumProgram()

# Creating registers
q = Q_program.create_quantum_register('q', 5)
c0 = Q_program.create_classical_register('c0', 1)
c1 = Q_program.create_classical_register('c1', 1)
c2 = Q_program.create_classical_register('c2', 1)

# Creates the quantum circuit
bit_flip = Q_program.create_circuit('bit_flip', [q], [c0,c1,c2])

# Prepares qubit in the desired initial state 
bit_flip.h(q[0])

# Encodes the qubit in a three-qubit entangled state  
bit_flip.cx(q[0], q[1])
bit_flip.cx(q[0], q[2])

# Go to Hadamard basis
bit_flip.h(q[0])
bit_flip.h(q[1])
bit_flip.h(q[2])

# Phase error on the second qubit
bit_flip.z(q[1])

# Converts phase error in bit-flip error
bit_flip.h(q[0])
bit_flip.h(q[1])
bit_flip.h(q[2])

# Adds additional two qubits for error-correction
bit_flip.cx(q[0], q[3])
bit_flip.cx(q[1], q[3])
bit_flip.cx(q[0], q[4])
bit_flip.cx(q[2], q[4])

# Measure the two additional qubits
bit_flip.measure(q[3], c0[0])
bit_flip.measure(q[4], c1[0])

# Do error correction
bit_flip.x(q[1]).c_if(c0, 1)
bit_flip.x(q[2]).c_if(c1, 1)

# Decodes the qubit from the three-qubit entangled state  
bit_flip.cx(q[0], q[1])
bit_flip.cx(q[0], q[2])

# Check the state of the initial qubit
bit_flip.measure(q[0], c2[0])

# Shows gates of the circuit
circuits = ['bit_flip']
print(Q_program.get_qasms(circuits)[0])

# Parameters for execution on simulator
backend = 'local_qasm_simulator' 
shots = 1024 # the number of shots in the experiment 

# Run the algorithm
result = Q_program.execute(circuits, backend=backend, shots=shots)

#Shows the results obtained from the quantum algorithm 
counts = result.get_counts('bit_flip')

print('\nThe measured outcomes of the circuits are:', counts)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c0[1];
creg c1[1];
creg c2[1];
h q[0];
cx q[0],q[1];
cx q[0],q[2];
h q[0];
h q[1];
h q[2];
z q[1];
h q[0];
h q[1];
h q[2];
cx q[0],q[3];
cx q[1],q[3];
cx q[0],q[4];
cx q[2],q[4];
measure q[3] -> c0[0];
measure q[4] -> c1[0];
if(c0==1) x q[1];
if(c1==1) x q[2];
cx q[0],q[1];
cx q[0],q[2];
measure q[0] -> c2[0];


The measured outcomes of the circuits are: {'0 0 1': 550, '1 0 1': 474}


## Exercises


<ol>
<li> 
Show the evolution of the state of the system for the transmission of the state $\lvert 1 \rangle$ through a noisy communication channel. Consider the case where:

<ol>
<li> 
A bit-flip error occurs to the first qubit of the encoded state
</li>

<li> 
A phase error occurs to the third qubit of the encoded state
</li>
</ol>

</li>


<li> 
Write a QISKit program that implements the exercises described above
</li>

</ol>

## References

[1] A. M. Steane, arXiv:quant-ph/0304016v2 (2003).
